## 1-23 Florian Rate XOR

In [ ]:
import json
import os
import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from spikey.logging import MetaReader
from spikey.viz import *

In [ ]:
def iter_epoch(dataframe, row_per_epoch):
    # Ceil division - 1
    for i in range((len(value) - 1) // row_per_epoch + 1 - 1):
        yield dataframe[i * row_per_epoch:(i + 1) * row_per_epoch]

In [ ]:
base = os.path.join('log')
FOLDERS = os.listdir(base)

dataset = {folder: MetaReader(folder=os.path.join(base, folder)) for folder in FOLDERS if os.path.isdir(os.path.join(base, folder))}

In [ ]:
for key, value in dataset.items():
    print(f"{key}: {value.summary}")

In [ ]:
## Violin plot of accuracies per ep
for key, value in dataset.items():
    N_AGENTS = value.summary['info']['population_config']['n_agents']
    N_REPEATS = value.summary['info']['metagame_info']['n_reruns']

    value = value.df

    ep_accuracies = []

    for i, episode in enumerate(iter_epoch(value, N_AGENTS)):
        ep_accuracies.append(np.copy(episode['fitness']))

    for i in range(len(ep_accuracies) // 10 - 1):
        plt.violinplot(ep_accuracies[i * 10:(i + 1) * 10])
        plt.title(f"Epoch {i * 10}-{(i + 1) * 10}")
        plt.show()

In [ ]:
## Scatter Accuracy & Entropy
for key, value in dataset.items():
    N_AGENTS = value.summary['info']['population_config']['n_agents']
    N_REPEATS = value.summary['info']['metagame_info']['n_reruns']
    value = value.df

    ep_accuracies = []

    X, Y_a = [], []

    for i, episode in enumerate(iter_epoch(value, N_AGENTS)):
        ep_accuracies.append(np.copy(episode['fitness']))

    for percentile in [50, 75, 90, 95, 100]:
        for i in range(len(ep_accuracies)):
            X.append(i)
            Y_a.append(np.percentile(ep_accuracies[i], percentile))

        plt.title(f"{percentile}th percentile of {key}")
        plt.scatter(X, Y_a, label="Accuracies")
        plt.legend()
        plt.show()

In [ ]:
## Scatter Accuracy & Entropy
for key, reader in dataset.items():
    df = reader.df

    params = df[df['fitness'] == np.max(df['fitness'])]
    filenames = df[df['fitness'] == np.max(df['fitness'])]['filename']
    
    print(np.max(df['fitness']))
    for i, filename in enumerate(filenames):
        print(params.iloc[i])
        #print(reader.read_file(filename)['accuracies'])